In [ ]:
%pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma gradio python-dotenv pypdf

In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the Google Drive path where your documents are stored
# google_drive_path = '/content/drive/MyDrive/local_rag/data'

In [25]:
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"

In [26]:
import os

# Get a list of all files in the specified folder
file_list = os.listdir(DATA_PATH)

# Filter the list to only include files with .pdf or .docx extensions
pdf_files = [f for f in file_list if f.endswith('.pdf')]
docx_files = [f for f in file_list if f.endswith('.docx')]

# Example: Print the list of PDF and DOCX files in the folder
print("PDF Files:", pdf_files)
print("DOCX Files:", docx_files)

PDF Files: ['AttentionIsAllyouNeed.pdf', 'bedrock-ug.pdf']
DOCX Files: []


In [27]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/avineetsharma/Project/Columbia/LocalDocuChat


In [28]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from uuid import uuid4
from langchain.embeddings import SentenceTransformerEmbeddings

# import the .env file
from dotenv import load_dotenv
load_dotenv()



True

In [ ]:
import os
import time
from uuid import uuid4
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory=CHROMA_PATH,
)


In [ ]:
def indexed_files(vector_store):
    stored_files = set(
        [
            meta["source"].split("/")[-1]
            for meta in vector_store._collection.get(include=["metadatas"])["metadatas"]
            if meta and "source" in meta
        ]
    )

    print ("stored_files:",stored_files)
    return stored_files

In [ ]:

def process_files(vector_store, embeddings_model):
    """
    Processes files in DATA_PATH, adding new files and removing deleted files.
    """
    current_files = set(os.listdir(DATA_PATH))

    print ("current_files:",current_files)

    stored_files = indexed_files(vector_store)


    # Find new files
    new_files = current_files - stored_files
    for filename in new_files:
        if filename.endswith(".pdf"):
            filepath = os.path.join(DATA_PATH, filename)
            loader = PyPDFLoader(filepath)
            raw_documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=300,
                chunk_overlap=100,
                length_function=len,
                is_separator_regex=False,
            )
            chunks = text_splitter.split_documents(raw_documents)
            print ("Document Split Complete",filename)
            uuids = [str(uuid4()) for _ in range(len(chunks))]
            vector_store.add_documents(documents=chunks, ids=uuids)
            print(f"Added file: {filename}")

    # Find deleted files
    deleted_files = stored_files - current_files
    for filename in deleted_files:
        if filename.endswith(".pdf"):
            filepath = os.path.join(DATA_PATH, filename)
            # Find document IDs associated with the deleted file
            results = vector_store._collection.get(
                where={"source": filepath}, include=["ids"]
            )
            if "ids" in results:
                ids_to_delete = results["ids"]
                vector_store._collection.delete(ids=ids_to_delete)
                print(f"Deleted file: {filename}")


def run_periodic_check():
    """
    Runs the periodic file check in a notebook environment.
    """
    embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
    vector_store = Chroma(
        collection_name="example_collection",
        embedding_function=embeddings_model,
        persist_directory=CHROMA_PATH,
    )
    process_files(vector_store, embeddings_model)
    # while False:
    #     process_files(vector_store, embeddings_model)
    #     time.sleep(interval_seconds)
    #     print(f"Checked for file changes. Next check in {interval_seconds} seconds...")



In [31]:
# To start the periodic check in your notebook, call this function:
run_periodic_check()

# To stop the check, you'll need to interrupt the kernel.

current_files: {'Nonviolent Communication_ A Language of Life_ Life-Changing Tools for Healthy Relationships ( PDFDrive ).pdf', '.DS_Store', 'AttentionIsAllyouNeed.pdf'}
stored_files: {'AttentionIsAllyouNeed.pdf'}
Document Split Complete Nonviolent Communication_ A Language of Life_ Life-Changing Tools for Healthy Relationships ( PDFDrive ).pdf
Added file: Nonviolent Communication_ A Language of Life_ Life-Changing Tools for Healthy Relationships ( PDFDrive ).pdf


In [ ]:
# # Load the embedding model
# embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# # Use the model with LangChain as before
# vector_store = Chroma(
#     collection_name="example_collection",
#     embedding_function=embeddings_model,
#     persist_directory=CHROMA_PATH,
# )

# # loading the PDF document
# raw_documents = []
# for filename in os.listdir(DATA_PATH):
#     if filename.endswith(".pdf"):
#         filepath = os.path.join(DATA_PATH, filename)
#         loader = PyPDFLoader(filepath)
#         raw_documents.extend(loader.load())

# # splitting the document
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=300,
#     chunk_overlap=100,
#     length_function=len,
#     is_separator_regex=False,
# )

# # creating the chunks
# chunks = text_splitter.split_documents(raw_documents)

# # creating unique ID's
# uuids = [str(uuid4()) for _ in range(len(chunks))]

# # adding chunks to vector store
# vector_store.add_documents(documents=chunks, ids=uuids)

In [33]:
num_results = 5

retriever = vector_store.as_retriever(search_kwargs={'k': num_results})

docs = retriever.invoke("What is communication")
print(type(docs[0].metadata))
print((docs[0].metadata["source"], docs[0].metadata["page_label"]))

<class 'dict'>
('data/Nonviolent Communication_ A Language of Life_ Life-Changing Tools for Healthy Relationships ( PDFDrive ).pdf', '22')
